## SUMMARY

Analysis of ligation products for spligation experiments.

Briefly, for RT-PCR Nanopore data, each product’s composition was confirmed by aligning reads to the corresponding amplicon with minimap2 (v2.28) and a custom junction file representing Csm cut sites. The abundance of each excision or ligation product was determined with HTSlib (v1.6)20, samtools (v1.6)21, and pysam (v0.18.0)22 and reported relative to total read depth within the crRNA target site.

RNA-seq data (GEO accession number GSE220741) were analyzed as previously described in Colognori et al. 2023. Ligation products are assessed below using the bam files published in the original paper.


In [1]:
#### IMPORT LIBS ####
import pandas as pd
from pandas.errors import EmptyDataError 
import numpy as np
import shutil
import os
import sys
import requests
import subprocess
import re
import glob
import pysamstats
import pysam
import Bio
from Bio.Seq import reverse_complement
from itertools import groupby
from operator import itemgetter
import matplotlib.pyplot as plt

from collections import Counter

## SUPRESS WARNINGS
import warnings
warnings.filterwarnings('ignore')

## SLURM
bash_header = ["#!/bin/bash", "#SBATCH -p standard", "#SBATCH --job-name ALIGN_SPLIG", 
               "#SBATCH -o %j.out", "#SBATCH -e %j.err", "## ACTIVATE CONDA", 
               'eval "$(conda shell.bash hook)"', 'conda activate Test_Py_3_9'
              ]


In [838]:
#### SETUP ####
'''
Configure strings corresponding to the crRNA target site.
Strings for ligation/deletion products will be used for display
'''

pwd = os.getcwd()
analysis_key = "Spligation_Key_12162025.xlsx"
sup_table_df = pd.read_excel("Spligation_Key_12162025.xlsx", sheet_name="Sup_Table_ROIs")

cut_indices = [3, 9, 15, 21] # Cut site locations
deltas = [24, 18, 12, 6] # Deletion sizes

# Target site strings
target_dict = {"LBR":["TACATCTACTAATGCTCTTCTGGCTTT", 27], # [String, length]
               "NCL":["AAGTTTGAATAGCTTCTGTCCCTCTGC", 27],
               "NPM1":["AAGTCTCTTTAAGAAAATAGTTTAAAC", 27]
}

# REF ROI COORDs
ref_roi_coords = pd.read_excel("Spligation_Key_12162025.xlsx", 
                               sheet_name = "Ref_ROI_Locations"
                              )
ligation_str_dict = {}

# ENDOGENOUS
for row in ref_roi_coords.itertuples():
    ref = row.Construct
    transcript = ref.split("_")[0]
    target_region = target_dict[transcript][0]
    print(f"## PROCESSING {transcript}: {target_region}")
    
    # LIGATION STRINGS
    cp_ends = ["".join(target_region[:n]) for n in cut_indices]
    oh_ends = ["".join(target_region[n:]) for n in cut_indices]+['']
    ligation_strs = [cp_end+(27-len(cp_end)-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
    ligation_strs = [x for x in ligation_strs if len(x)==27 and "-" in x]
    print('\n'.join(ligation_strs))
    
    ligation_str_dict[transcript] = ligation_strs
    
# 1 CUT Cis: 'Cis_1Cut'
reporter_target_region = 'GCGGATCAAGTTCTGTAGCAGTGTTCAAGTGG'
cut_indices = [3, 9, 15, 21, 27] # Plot full target region (32nt)
cp_ends = ["".join(reporter_target_region[:n]) for n in cut_indices]
oh_ends = ["".join(reporter_target_region[n:]) for n in cut_indices]
ligation_strs = [cp_end+(32-len(cp_end)-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
ligation_strs = [x for x in ligation_strs if len(x)==32 and "-" in x]+[reporter_target_region]
print('\n'.join(ligation_strs))
ligation_str_dict['Cis_1Cut'] = ligation_strs

# 2 CUT Cis: '2Cut_Short_Cis_Amplicon', '2Cut_Long_Cis_Amplicon'
large_del_conversion_dict = {}
n_term = "GCGGATCAAGTTCTGTCGCAGTGTTCAAGTGG"
cp_ends = ["".join(n_term[:n]) for n in cut_indices]
oh_ends = ["".join(n_term[n:]) for n in cut_indices]
formatted_large_del_ligation_strs = [cp_end+(27-len(cp_end))*"-"+"-----|"\
                 +"---"\
                 +(29-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]

print('\n'.join(large_del_ligation_strs))
large_del_conditions = ['Cis_2CutShort', 'Cis_2CutLong']

for large_del_condition in large_del_conditions:
    del_row = sup_table_df.loc[sup_table_df.Analysis == large_del_condition]
    del_start = int(del_row.ROI_Skip_Start.values[0])
    del_stop = int(del_row.ROI_Skip_Stop.values[0])
    del_str = (del_stop-del_start)*"-"
    large_del_ligation_strs = [cp_end+(27-len(cp_end))*"-"+"|"\
                 +(29-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
    temp_lig_strs = [x.replace('|', f'{del_str}') for x in large_del_ligation_strs]
    ligation_str_dict[large_del_condition] = temp_lig_strs
    large_del_conversion_dict[large_del_condition] = dict(zip(temp_lig_strs,formatted_large_del_ligation_strs))

# TRANS
n_term = "GCGGATCAAGTTCTGTCGCAGTGTTCAAGTGG"
cp_ends = ["".join(n_term[:n]) for n in cut_indices]
oh_ends = ["".join(n_term[n:]) for n in cut_indices]
trans_del_ligation_strs = [cp_end+(27-len(cp_end))*"-"+
                 +(29-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
clean_pipe = [x.replace('|', f'') for x in trans_del_ligation_strs]
ligation_str_dict['Trans_Basic'] = clean_pipe
trans_conversion_dict = dict(zip(clean_pipe,formatted_large_del_ligation_strs))

# XIST
xist_conditions = ['XIST_crNA_1', 'XIST_crNA_2']
xist_conversion_dict = {}
for xist_condition in xist_conditions:
    print(f"##{xist_condition}")
    xist_row = sup_table_df.loc[sup_table_df.Analysis == xist_condition]
    crrna_str = xist_row.ROI_Str.values[0]
    target_site = str(Bio.Seq.Seq(crrna_str).reverse_complement())
    
    print("# TARGET SITE: ",target_site)
    cut_indices = [3, 9, 15, 21]
    deltas = [24, 18, 12, 6]

    cut_indices = [3, 9, 15, 21, 27] # Plot full target region (32nt)
    cp_ends = ["".join(target_site[:n]) for n in cut_indices]
    oh_ends = ["".join(target_site[n:]) for n in cut_indices]
    ligation_strs = [cp_end+(32-len(cp_end)-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
    ligation_strs = [x for x in ligation_strs if len(x)==32 and "-" in x]+[target_site]
    ligation_str_dict[xist_condition] = ligation_strs
    
    conversion_strs = [str(Bio.Seq.Seq(x).reverse_complement()) for x in ligation_strs]
    xist_conversion_dict[xist_condition] = dict(zip(conversion_strs,ligation_strs))

## PROCESSING LBR: TACATCTACTAATGCTCTTCTGGCTTT
TAC------TAATGCTCTTCTGGCTTT
TAC------------TCTTCTGGCTTT
TAC------------------GGCTTT
TAC------------------------
TACATCTAC------TCTTCTGGCTTT
TACATCTAC------------GGCTTT
TACATCTAC------------------
TACATCTACTAATGC------GGCTTT
TACATCTACTAATGC------------
TACATCTACTAATGCTCTTCT------
## PROCESSING NCL: AAGTTTGAATAGCTTCTGTCCCTCTGC
AAG------TAGCTTCTGTCCCTCTGC
AAG------------CTGTCCCTCTGC
AAG------------------CTCTGC
AAG------------------------
AAGTTTGAA------CTGTCCCTCTGC
AAGTTTGAA------------CTCTGC
AAGTTTGAA------------------
AAGTTTGAATAGCTT------CTCTGC
AAGTTTGAATAGCTT------------
AAGTTTGAATAGCTTCTGTCC------
## PROCESSING NPM1: AAGTCTCTTTAAGAAAATAGTTTAAAC
AAG------TAAGAAAATAGTTTAAAC
AAG------------AATAGTTTAAAC
AAG------------------TTAAAC
AAG------------------------
AAGTCTCTT------AATAGTTTAAAC
AAGTCTCTT------------TTAAAC
AAGTCTCTT------------------
AAGTCTCTTTAAGAA------TTAAAC
AAGTCTCTTTAAGAA------------
AAGTCTCTTTAAGAAAATAGT------
GCG------GTTCTGTAG

In [840]:
#### PROCESS REFERENCES ####
'''
Create splice junction file corresponding to 6nt cut sites characteristic of CRISPR-Csm
'''

contig_junction_dict = {}
start_stop_str_dicts = {}

for idx,roi in ref_roi_coords.iterrows():
    construct = roi['Construct']
    target_starts = [int(x) for x in roi[['Target1_0Based_Start', 'Target2_0Based_Start']].values if str(x) != "nan"]

    seq = target_dict[construct.split('_')[0]][0]
    max_str_len = target_dict[construct.split('_')[0]][-1]

    
    # LIGATION STRINGS
    cp_ends = ["".join(seq[:n]) for n in cut_indices]
    oh_ends = ["".join(seq[n:]) for n in cut_indices]+['']
    len_cp_ends = [len(cp_end) for cp_end in cp_ends]
    len_oh_ends = [len(oh_end) for oh_end in oh_ends]
    ligation_strs = [cp_end+(27-len(cp_end)-len(oh_end))*"-"+oh_end for cp_end in cp_ends for oh_end in oh_ends]
    ligation_strs = [x for x in ligation_strs if len(x)==27 and "-" in x]
    print(construct,seq, cp_ends)
    
    # DEFINE JUNCTION SITES
    ligation_locs = [f'{len(cp_end)+target_starts[0]}__{len(cp_end)+target_starts[0]+(max_str_len-len(cp_end)-len(oh_end))}' for cp_end in cp_ends for oh_end in oh_ends]
    start_stop_str_dict = dict(zip(ligation_locs, ligation_strs))
    start_stop_str_dicts[construct] = start_stop_str_dict
    print(len_cp_ends, len_oh_ends, f"!!!!{len(ligation_strs)}")
    print("\n".join([f"{ligation_str}: {len(ligation_str)}, {ligation_locs[idx]}" for idx,ligation_str in enumerate(ligation_strs)]))

    # SAVE BED6
    bed6_lines = []
    strand = "+"
    temp_dict = {}
    
    for i, loc in enumerate(ligation_locs):
        start, end = loc.split("__")
        bed6_lines.append(f"{construct.replace('.fasta','')}\t{start}\t{end}\tjunc{i+1}\t0\t{strand}")
        temp_dict[f"junc{i+1}"] = loc
    contig_junction_dict[construct]= temp_dict

    # Save to file
    with open(f"References/{construct.replace('.fasta','')}_junctions.bed", "w") as f:
        for line in bed6_lines:
            f.write(line + "\n")

LBR_Amplicon_With_Target_Site.fasta TACATCTACTAATGCTCTTCTGGCTTT ['TAC', 'TACATCTAC', 'TACATCTACTAATGC', 'TACATCTACTAATGCTCTTCT', 'TACATCTACTAATGCTCTTCTGGCTTT']
[3, 9, 15, 21, 27] [24, 18, 12, 6, 0, 0] !!!!14
TAC------TAATGCTCTTCTGGCTTT: 27, 466__466
TAC------------TCTTCTGGCTTT: 27, 466__472
TAC------------------GGCTTT: 27, 466__478
TAC------------------------: 27, 466__484
TAC------------------------: 27, 466__490
TACATCTAC------TCTTCTGGCTTT: 27, 466__490
TACATCTAC------------GGCTTT: 27, 472__466
TACATCTAC------------------: 27, 472__472
TACATCTAC------------------: 27, 472__478
TACATCTACTAATGC------GGCTTT: 27, 472__484
TACATCTACTAATGC------------: 27, 472__490
TACATCTACTAATGC------------: 27, 472__490
TACATCTACTAATGCTCTTCT------: 27, 478__466
TACATCTACTAATGCTCTTCT------: 27, 478__472
NCL_Amplicon_With_Target_Site.fasta AAGTTTGAATAGCTTCTGTCCCTCTGC ['AAG', 'AAGTTTGAA', 'AAGTTTGAATAGCTT', 'AAGTTTGAATAGCTTCTGTCC', 'AAGTTTGAATAGCTTCTGTCCCTCTGC']
[3, 9, 15, 21, 27] [24, 18, 12, 6, 0, 0] !!!

In [841]:
#### SETUP ALIGNMENTS ####
'''
Align Nanopore data to amplicon from RT-PCR experiments
'''

threads = "$SLURM_CPUS_ON_NODE"
bash_lines = bash_header
bash_script_name = "Nanopore_Alignments.sh"

alignment_key = pd.read_excel("Spligation_Key_12162025.xlsx", sheet_name="Alignment_Key")


data_dir = f"{pwd}/Data/"
ref_dir = f"{pwd}/References/"
alignment_dir = f"{pwd}/Alignments/"

fastas = glob.glob(f"References/*.fasta")
juncs = glob.glob(f"References/*junctions.bed")
fastqs = glob.glob(f"{pwd}/Data/*.fastq")

qcd = []
bams_1ref_basic = {}
bams_1ref_splice = {}
for idx,row in alignment_key.iterrows(): 
    analysis_id = row['Sample_Description'].replace(' ', "_")
    fq = f'{data_dir}{row["FASTQ_Name"]}'
    sample_id = f"{analysis_id}__" + fq.split("/")[-1].split(".")[0]
    ref_1seq = f"{ref_dir}{row.Reference_for_Alignment}.fasta"
    
    print(f"## PROCESSING: {sample_id}")
    print(fq)
    print(ref_1seq)
    

    bash_lines.append(f'#### {analysis_id}')
    
    # QC
    qc_dir = data_dir
    if fq not in qcd:
        bash_lines.append("# QC")
        qc_str = f"NanoPlot -t {threads} --fastq {fq} --maxlength 10000 --plots dot --prefix {sample_id} --outdir {qc_dir}"
        bash_lines.append(qc_str)
        qcd.append(fq)
        
    
    # 1REF SPLICE MODE ALIGNMENT
    bash_lines.append("# 1REF SPLICE MODE ALIGNMENT")
    bam_1ref_splice = f"{alignment_dir}{sample_id}__1ref_splice_sorted.bam"
    bams_1ref_splice[sample_id] = bam_1ref_splice
    align_1ref_splice = (f"minimap2 -ax splice -k12 -t {threads} -J 1 {ref_1seq} {fq} "
                         f"| samtools view -bS -@ {threads} - "
                         f"| samtools sort -@ {threads} -o {bam_1ref_splice} - && "
                         f"samtools index  -@ {threads} {bam_1ref_splice}"
                        )
    bash_lines.append(align_1ref_splice)

with open(bash_script_name, "w") as f:
    f.writelines("\n".join(bash_lines))


#!sbatch Nanopore_Alignments.sh

## PROCESSING: LBR__D9S6PX_1_EndoDeluxe-LBR
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Data/D9S6PX_1_EndoDeluxe-LBR.fastq
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/References/LBR_Amplicon_With_Target_Site.fasta
## PROCESSING: NCL__D9S6PX_2_EndoDeluxe-NCL
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Data/D9S6PX_2_EndoDeluxe-NCL.fastq
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/References/NCL_Amplicon_With_Target_Site.fasta
## PROCESSING: NPM1__D9S6PX_3_EndoDeluxe-NPM1
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Data/D9S6PX_3_EndoDeluxe-NPM1.fastq
/groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/References/NPM1_Amplicon_With_Target_Site.fasta


In [842]:
#### SETUP BIGWIG COVERAGE TRACKS
'''
Create bigwigs for IGV coverage tracks
'''
threads = "$SLURM_CPUS_ON_NODE"
bash_lines = bash_header
bash_script_name = "Bigwigs.sh"

for sample_id, bam in bams_1ref_splice.items():  
    print(sample_id)
    ## BIGWIG
    bash_lines.append(f"# BigWig")
    bw_str = f"bamCoverage -b {bam} -o {bam.replace('.bam', '.bw')} --numberOfProcessors {threads} --binSize 1"
    bash_lines.append(bw_str)
    
    bash_lines.append(f"# CPM BigWig")
    bw_cpm_str = f"bamCoverage -b {bam} -o {bam.replace('.bam', '_CPM.bw')} --normalizeUsing CPM --numberOfProcessors {threads} --binSize 1"
    bash_lines.append(bw_cpm_str)
    print(bw_str, bw_cpm_str)

with open(bash_script_name, "w") as f:
    f.writelines("\n".join(bash_lines))

#!sbatch Bigwigs.sh

LBR__D9S6PX_1_EndoDeluxe-LBR
bamCoverage -b /groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Alignments/LBR__D9S6PX_1_EndoDeluxe-LBR__1ref_splice_sorted.bam -o /groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Alignments/LBR__D9S6PX_1_EndoDeluxe-LBR__1ref_splice_sorted.bw --numberOfProcessors 16 --binSize 1 bamCoverage -b /groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Alignments/LBR__D9S6PX_1_EndoDeluxe-LBR__1ref_splice_sorted.bam -o /groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Alignments/LBR__D9S6PX_1_EndoDeluxe-LBR__1ref_splice_sorted_CPM.bw --normalizeUsing CPM --numberOfProcessors 16 --binSize 1
NCL__D9S6PX_2_EndoDeluxe-NCL
bamCoverage -b /groups/doudna/projects/mtrinidad_projects/Csm_RNASeq_David/Spligation/Nanopore_12182025/Alignments/NCL__D9S6PX_2_EndoDeluxe-NCL__1ref_splice_sorted.bam -o /groups/doudna/projects/mtri

In [843]:
#### TABULATE ENDOGENOUS LIGATION: sfGFP #####
'''
Tabulate ligation products for endogenous spligation of C-term sfGFP
'''

final_count_dict = {}
full_span_dict = {}
for alignment_id, bam_file in bams_1ref_splice.items():
    ref_id = alignment_id.split("__")[0]
    ref = alignment_key.loc[
        alignment_key.Sample_Description == ref_id
    ].Reference_for_Alignment.values[0]

    print(f"#### Processing {ref_id}")
    bam = pysam.AlignmentFile(bam_file, "rb")

    start = ref_roi_coords[
        ref_roi_coords.Construct.str.startswith(ref)
    ].Target1_0Based_Start.values[0]
    end = start + 27

    counts = Counter()

    for read in bam.fetch(ref, start, end):
        if read.is_unmapped:
            continue

        # FULL-SPAN REQUIREMENT
        if read.reference_start > start or read.reference_end < end:
            continue

        aligned = []
        ref_pos = read.reference_start
        query_pos = 0

        for op, length in read.cigartuples:

            # M, =, X: matches
            if op in (0, 7, 8):
                for _ in range(length):
                    if start <= ref_pos < end:
                        aligned.append(read.query_sequence[query_pos])
                    ref_pos += 1
                    query_pos += 1

            # I: insertions
            elif op == 1:
                if start <= ref_pos < end:
                    aligned.append(read.query_sequence[query_pos:query_pos + length])
                query_pos += length

            # D: deletions
            elif op in (2,3):
                for _ in range(length):
                    if start <= ref_pos < end:
                        aligned.append("-")
                    ref_pos += 1

            # Soft clip
            elif op == 4:
                query_pos += length

            # Hard clip
            elif op == 5:
                pass

        seq = "".join(aligned)

        # Increment string count
        if len(seq) > 0:
            counts[seq] += 1

    for seq, count in counts.most_common():
        print(count, seq)

    final_count_dict[ref_id] = counts
    
    # COUNT TOTAL FULL-SPAN READS
    full_span_count = 0

    for read in bam.fetch(ref, start, end):
        if read.is_unmapped:
            continue
        if read.reference_start <= start and read.reference_end >= end:
            full_span_count += 1
            
    full_span_dict[alignment_id.split("_")[0]] = full_span_count
    

#### Processing LBR
3153 TACATCTAC------------------
549 TACATCTACTAATGCTCTTC------T
14 CCGCCATGGCCGAGGAAGGCATTGCTGCTGGAGGTGT
10 ---ATGT-CTCCCCCTCCTTCTAGG--GG
9 ---------GTGGTGGCCGGGGCATGGACCGAGGTGGCTTTGGT
9 GACATCTAC------------------
5 TACATCTA-------------------
3 TAC--GTAC------------------
3 TCACATCTAC------------------
3 TACATCGACGAATGCTCTTC------T
2 TACATCTACTAATGCTC--------TT
2 TACATC--------------------T
2 TACATCTAC--------------C---
2 ----TGTAGCCTGTTGCCCATCCCCAGG---G
2 TACATCTACCAA----------G----
2 TACGTCT-CTAATGCTCTTC------T
2 TACATC---------------------
2 TACAT-TAC------------------
2 TACA----------------------T
2 -ACATCTAC------------------
2 GACATC----AA---------------
1 TACATATACTA------------C---
1 TACAT----------------------
1 TACATCTACCAA---------GG----
1 ---------TCCATGTCTCCCCTTGTCAGG--GG
1 TACATCTGCCAA----------G----
1 TACATTTATTAATGCTCTTC------T
1 TAC--CTACGTA-----CTT---AC---
1 TACATCTAC----------C-GG----
1 TACATCTCTTAATGCTCTTC------T
1 TA-----AC----------C---C---


In [844]:
###################################### TABULATE PRIOR LIGATIONS ######################################
sup_table_df = pd.read_excel("Spligation_Key_12162025.xlsx", sheet_name="Sup_Table_ROIs")
final_count_dict_prior = {}
full_span_dict_prior = {}

In [845]:
#### TABULATE Prior: normal amplicon Ref ####
'''
Tabulate ligation products for prior XIST and reporter experiments
'''
hg38_processing = ['XIST_crNA_1', 'XIST_crNA_2']
normal_processing = ['Cis_1Cut', 'Trans_Basic'] + hg38_processing

normal_processing_df = sup_table_df.loc[sup_table_df.Analysis.isin(normal_processing)]

final_count_dict_normal_prior = {}

for row in normal_processing_df.itertuples():
    alignment_id = row.Analysis
    bam_file = row.BAM
    chromo = row.chr
    roi_str = row.ROI_Str

    print(f"#### Processing {alignment_id}")
    print(f"\t{roi_str}")
    bam = pysam.AlignmentFile(bam_file, "rb")

    start = row.ROI_Start
    end = start + len(roi_str)

    counts = Counter()

    for read in bam.fetch(chromo, start, end):
        # SKIP UNMAPPED READS
        if read.is_unmapped:
            continue

        # FULL-SPAN REQUIREMENT
        if read.reference_start > start or read.reference_end < end:
            continue

        aligned = []
        ref_pos = read.reference_start
        query_pos = 0


        for op, length in read.cigartuples:
            # M, =, X: matches
            if op in (0, 7, 8):
                for i in range(length):
                    if start <= ref_pos < end:
                        aligned.append(read.query_sequence[query_pos])
                    ref_pos += 1
                    query_pos += 1

            # I: insertions
            elif op == 1:
                if start <= ref_pos < end:
                    aligned.append(read.query_sequence[query_pos:query_pos+length])
                query_pos += length

            # D: deletions
            elif op in (2, 3):
                for i in range(length):
                    if start <= ref_pos < end:
                        aligned.append("-")
                    ref_pos += 1

            # Soft clip
            elif op == 4:
                query_pos += length

            # Hard clip (ignore)
            elif op == 5:
                pass

        seq = "".join(aligned)

        # Increment count
        if len(seq) > 0:
            counts[seq] += 1

    final_count_dict_prior[alignment_id] = counts
    
    
    # COUNT TOTAL FULL-SPAN READS
    full_span_count = 0

    for read in bam.fetch(chromo, start, end):
        if read.is_unmapped:
            continue
        if read.reference_start <= start and read.reference_end >= end:
            full_span_count += 1
            
    full_span_dict_prior[alignment_id] = full_span_count

#### Processing XIST_crNA_1
	GCCACTTGAACACTGCGACAGAACTGGATCCG
#### Processing XIST_crNA_2
	TTGGACAACCTAACAAAGCACAGCCCGCCATG
#### Processing Cis_1Cut
	GCGGATCAAGTTCTGTAGCAGTGTTCAAGTGG
#### Processing Trans_Basic
	GCGGATCAAGTTCTGTCGCAGTGTTCAGATCAAGTTCTGTCGCAGTGTTCAAGTGG


In [846]:
#### PROCESS PRIOR: Ref with deletion ####
'''
Tabulate ligation products for 2-cut reporters with longer ROIs
(as defined by "end" variable in line 22)
'''
del_processing = ['Cis_2CutShort', 'Cis_2CutLong']

del_processing_df = sup_table_df.loc[sup_table_df.Analysis.isin(del_processing)]

final_count_dict_del_prior = {}

for row in del_processing_df.itertuples():
    alignment_id = row.Analysis
    bam_file = row.BAM
    chromo = row.chr
    roi_str = row.ROI_Str

    print(f"#### Processing {alignment_id}")
    print(f"\t{roi_str}")
    bam = pysam.AlignmentFile(bam_file, "rb")

    start = row.ROI_Start
    end = row.ROI_Stop+1

    counts = Counter()

    for read in bam.fetch(chromo, start, end):
        # SKIP UNMAPPED READS
        if read.is_unmapped:
            continue

        # FULL-SPAN REQUIREMENT
        if read.reference_start > start or read.reference_end < end:
            continue

        aligned = []
        ref_pos = read.reference_start
        query_pos = 0


        for op, length in read.cigartuples:
            # M, =, X : Matches
            if op in (0, 7, 8):
                for i in range(length):
                    if start <= ref_pos < end:
                        aligned.append(read.query_sequence[query_pos])
                    ref_pos += 1
                    query_pos += 1

            # I: insertions
            elif op == 1:
                if start <= ref_pos < end:
                    aligned.append(read.query_sequence[query_pos:query_pos+length])
                query_pos += length

            # D: deletions
            elif op in (2, 3):
                for i in range(length):
                    if start <= ref_pos < end:
                        aligned.append("-")
                    ref_pos += 1

            # Soft clip
            elif op == 4:
                query_pos += length

            # Hard clip (ignore)
            elif op == 5:
                pass

        seq = "".join(aligned)


        if len(seq) > 0:
            counts[seq] += 1

    final_count_dict_prior[alignment_id] = counts
    
    
    # COUNT TOTAL FULL-SPAN READS
    full_span_count = 0

    for read in bam.fetch(chromo, start, end):
        if read.is_unmapped:
            continue
        if read.reference_start <= start and read.reference_end >= end:
            full_span_count += 1
            
    full_span_dict_prior[alignment_id] = full_span_count

final_count_dict.update(final_count_dict_prior)
full_span_dict.update(full_span_dict_prior)

#### Processing Cis_2CutShort
	GCGGATCAAGTTCTGTCGCAGTGTTCAGATCAAGTTCTGTCGCAGTGTTCAAGTGG
#### Processing Cis_2CutLong
	GCGGATCAAGTTCTGTCGCAGTGTTCAGATCAAGTTCTGTCGCAGTGTTCAAGTGG


In [847]:
#### SAVE ####
'''
Configure counts with display strings and save to Excel file
'''

# Adjust strings for minor alignment artifacts
fix_strs = {'LBR':{'TACATCTACTAATGCTCTTC------T':'TACATCTACTAATGCTCTTCT------'},
            'NCL':{},
            'NPM1':{},
            'Cis_1Cut':{'GCGGATCA------------------AAGTGG':'GCGGATCAA------------------AGTGG'},
            'Cis_2CutShort':{}, 
            'Cis_2CutLong':{},
            'Trans_Basic':{},
            'XIST_crNA_1':{},
            'XIST_crNA_2':{}
            
            
}

## COLLECT DATA
dfs = {}
for sample_id, data in final_count_dict.items():
    
    print(f"#### PROCESSING: {sample_id}")
    
    # FIX
    fix_dict = fix_strs[sample_id]
    for bad_str,new_str in fix_dict.items():
        new_value = data[bad_str] + data[new_str]
        data[new_str] = new_value
    
    # FILTER DICT FOR KNOWN LIGATION PRODUCTS
    if 'XIST' in sample_id: # If Xist, convert to standardize target-site str format
        temp_converter = xist_conversion_dict[sample_id]
        data_clean = {temp_converter[k]:v for k,v in data.items() if k in temp_converter.keys()}
        data_clean.update({k:0 for k in ligation_str_dict[sample_id] if k not in data_clean.keys()})
    elif '2Cut' in sample_id:
        data.update({k:0 for k in ligation_str_dict[sample_id] if k not in data.keys()})
        temp_converter = large_del_conversion_dict[sample_id]
        data_clean = {temp_converter[k]:v for k,v in data.items() if k in temp_converter.keys()}
        data_clean.update({k:0 for k in ligation_str_dict[sample_id] if k not in data_clean.keys()})
    elif "Trans" in sample_id:
        data.update({k:0 for k in ligation_str_dict[sample_id] if k not in data.keys()})
        temp_converter = trans_conversion_dict
        data_clean = {temp_converter[k]:v for k,v in data.items() if k in temp_converter.keys()}
    else:
        data.update({k:0 for k in ligation_str_dict[sample_id] if k not in data.keys()})
        data_clean = {k:v for k,v in data.items() if k in ligation_str_dict[sample_id]}
    
    
    n_spanning_reads = full_span_dict[sample_id]

    data_clean['Other_Reads'] = n_spanning_reads - sum(data_clean.values())
    data_clean['Total_Reads'] = n_spanning_reads
    
    # Create DF
    df = pd.DataFrame({'Ligation_Product':data_clean.keys(),
                       'N_Reads':data_clean.values()
                      })
    
    df['Percent'] = 100*df['N_Reads']/n_spanning_reads
    dfs[sample_id] = df

    
## SAVE
with pd.ExcelWriter("Junction_Counts_12222025.xlsx", engine="openpyxl") as writer:
    for sample_id, df in dfs.items():
        if df.empty:
            continue

        # Save to Excel sheet
        sheet_name = sample_id[:31] # 31 chars max
        df.to_excel(writer, sheet_name=sheet_name, index=False)

#### PROCESSING: LBR
#### PROCESSING: NCL
#### PROCESSING: NPM1
#### PROCESSING: XIST_crNA_1
#### PROCESSING: XIST_crNA_2
#### PROCESSING: Cis_1Cut
#### PROCESSING: Trans_Basic
#### PROCESSING: Cis_2CutShort
#### PROCESSING: Cis_2CutLong
